In [1]:
!python -m pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\python.exe -m pip install --upgrade pip' command.


In [3]:
from sys import float_info

import numpy as np

def gaussian_func(x: np.ndarray, theta: float, mu: float, sigma: float):
    return theta * np.exp(-np.power((x - mu) / sigma, 2) / 2.0)

def approx_func(x: np.ndarray, theta: np.ndarray, mu: np.ndarray,
        sigma: np.ndarray):
    k = len(theta)
    assert k == len(mu) == len(sigma)
    result = np.zeros(len(x))
    for i in range(k):
        result += gaussian_func(x, theta[i], mu[i], sigma[i])

def func(params: np.ndarray, *args) -> float:
    theta, mu, sigma = np.split(params, 3)
    assert len(theta) == len(mu) == len(sigma)
    x, y = args
    diff = y - approx_func(x, theta, mu, sigma)
    return np.sum(np.power(diff, 2))

def func_jac(params: np.ndarray, *args) -> float:
    theta, mu, sigma = np.split(params, 3)
    k = len(theta)
    assert k == len(mu) == len(sigma)
    x, y = args
    diff = y - approx_func(x, theta, mu, sigma)
    result = np.full(3 * k, -2.0)
    deriv_theta, deriv_mu, deriv_sigma = np.split(result, 3)
    for i in range(k):
        deriv_theta[i] *= np.sum(diff * gaussian_func(x, 1.0, mu[i], sigma[i]))
        temp = diff * gaussian_func(x, theta[i], mu[i], sigma[i])
        deriv_mu[i] *= np.sum(temp * (x - mu[i]) / np.power(sigma[i], 2))
        deriv_mu[i] *= np.sum(
            temp * np.power(x - mu[i], 2) / np.power(sigma[i], 3))
    return result
"""
def func_hess(params: np.ndarray, *args) -> float:
    k = len(params) // 3
    theta, mu, sigma = np.split(params, 3)
    assert k == len(theta) == len(mu) == len(sigma)
    x, y = args
    delta = y - gaussian_func(x, k, theta, mu, sigma)
    return np.sum(np.power(delta))
"""
def lsq_gaussian(x: np.ndarray, y: np.ndarray, k: int, method: str,
        epsilon: float = 1E-7, max_iter: int = 100) -> np.ndarray:
    if (not isinstance(x, np.ndarray) or not isinstance(y, np.ndarray)
            or not isinstance(k, int) or not isinstance(method, str)
            or not isinstance(epsilon, float) or not isinstance(max_iter, int)):
        raise TypeError()
    n = len(x)
    if (len(x.shape) != 1 or n < 2 or x.shape != y.shape or k < 1
            or epsilon < float_info.epsilon or max_iter < 1):
        raise ValueError()
    x0 = np.full(3 * k, 1.0)
    result = minimize(fun=func, x0=x0, args=(x, y),
        method=method, jac=func_jac, tol=epsilon) # hess=func_hess, options={'maxiter': max_iter})
    return result

NameError: name 'nd' is not defined

In [ ]:
from math import cos, sin

def f(x: float) -> float:
    return 1.0 + 1.5 * x + 2.0 * cos(3.0 * x) + 3.0 * sin(7.0 * x)

start, stop = 0.0, 4.0
x = np.linspace(start, stop, 100)
f_vectorize = np.vectorize(f)
y = f_vectorize(x)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y, color='red', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
from numpy.random import normal

n = len(x)
y_noise = y + normal(loc=0.0, scale=1.25, size=n)
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
result = lsq_gaussian(x, y_noise, 4, 'Nelder-Mead')
theta, nu, sigma = np.split(result.x, 3)
approx = approx_func(x, theta, nu, sigma)

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
subplot.plot(x, approx, color='blue', lw=2, label='approx')
plt.legend()
plt.show()

In [ ]:
k = 8
params = np.zeros(k * 3)
step = (start - stop) / k
for i in range(k):
    params[i] = np.sum(x[i * k : k * (i + 1)]) / k
    params[k + i] = start + (i + 0.5) * step
params[2 * k : 3 * k] = step / 2
params[1] *= -1

theta = params[: k]
mu = params[k : 2 * k]
sigma = params[2 * k :]

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, approx_func(x, k, theta, mu, sigma), 'Xr', label='approx')
# subplot.plot(x, y, color='green', lw=2, label='y')
plt.legend()
plt.show()
print(theta)
print(mu)
print(sigma)